In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend

In [2]:
train_dir = os.path.dirname('/root/Downloads/dog-cat/train/')
val_dir = os.path.dirname('/root/Downloads/dog-cat/val_data/')


In [3]:
img_width = 256
img_height = 256


In [4]:

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,classes=['cat', 'dog'], class_mode = 'binary',batch_size = 50)

Found 20000 images belonging to 2 classes.


In [5]:
val_generator = train_datagen.flow_from_directory(val_dir,class_mode = 'binary',batch_size = 32)


Found 5000 images belonging to 2 classes.


In [6]:
model = Sequential()
model.add(Convolution2D(32,3,3,input_shape = (img_width,img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(32,3,3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(64,3,3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))


In [7]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [8]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000,
        epochs=20,
        validation_data=val_generator,
        validation_steps=800)


Epoch 1/20
2000/2000 [==============================] - 509s 255ms/step - loss: 0.5994 - accuracy: 0.6686 - val_loss: 0.5259 - val_accuracy: 0.7374
Epoch 2/20
2000/2000 [==============================] - 435s 217ms/step - loss: 0.4841 - accuracy: 0.7636 - val_loss: 0.5623 - val_accuracy: 0.7146
Epoch 3/20
2000/2000 [==============================] - 424s 212ms/step - loss: 0.3988 - accuracy: 0.8162 - val_loss: 0.5643 - val_accuracy: 0.7464
Epoch 4/20
2000/2000 [==============================] - 438s 219ms/step - loss: 0.3213 - accuracy: 0.8587 - val_loss: 0.5991 - val_accuracy: 0.7439
Epoch 5/20
2000/2000 [==============================] - 450s 225ms/step - loss: 0.2540 - accuracy: 0.8921 - val_loss: 0.7636 - val_accuracy: 0.7424
Epoch 6/20
2000/2000 [==============================] - 431s 215ms/step - loss: 0.1948 - accuracy: 0.9197 - val_loss: 0.9865 - val_accuracy: 0.7184
Epoch 7/20
2000/2000 [==============================] - 427s 214ms/step - loss: 0.1488 - accuracy: 0.9406 - val_

In [25]:
# import h5py
# model.save_weights('models/cnn.h5')

from tensorflow.keras.models import model_from_json
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("model.h5")
print("Saved model to disk")

In [26]:
from tensorflow.keras.models import model_from_json


In [27]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [28]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [40]:
test_dir = os.path.dirname('/root/Downloads/dog-cat/test/')
test_generator = train_datagen.flow_from_directory(test_dir,class_mode = 'binary',batch_size = 32)


Found 12500 images belonging to 1 classes.


In [42]:
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
predict = loaded_model.predict_generator(test_generator, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)


In [45]:
print(predict)

[[1.0000000e+00]
 [5.1931083e-01]
 [5.9604645e-08]
 ...
 [1.0000000e+00]
 [9.9997437e-01]
 [8.1960142e-01]]


In [47]:
import pandas as pd
sub = pd.read_csv("/root/Downloads/dog-cat/sampleSubmission.csv")
sub.head()

,id,label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [49]:
sub.label = predict
sub.head()

,id,label
0,1,1.000000e+00
1,2,5.193108e-01
2,3,5.960464e-08
3,4,1.000000e+00
4,5,9.988003e-01


In [50]:
sub.to_csv("/root/Downloads/dog-cat/sampleSubmission.csv",index=False)